In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as an

In [5]:
class Base(object):
    def __init__(self, r0 = 300000, deltaT = 1, fmax=1, mass = 1):
        self.cg = 3.9886e14
        self.rknot = r0
        self.mu1 = np.sqrt(self.cg/self.rknot**3)
        print(self.mu1, self.cg, self.rknot)
        self.deltaT = deltaT
        ndelt = self.mu1*deltaT
        sin = np.sin(self.mu1*deltaT )
        cos = np.cos(self.mu1*deltaT )
        self.fmax = fmax
        self.mass = mass

        self.Ad = np.array([[4-3*cos,           0,      1/self.mu1*cos,         2/self.mu1*(1-cos)          ],
                            [6*(sin-ndelt),     1,      -2/self.mu1*(1-cos),     1/self.mu1*(4*sin -3 *ndelt)],
                            [3*self.mu1*sin,    0,      cos,                    2*sin],
                            [6*(1-cos),        0,      -2*sin,                 4*cos-3]])


        invn = 1/self.mu1
        insqr = self.mu1**(-2)
        self.Bd = fmax/mass * np.array([[insqr*(1-cos),         2*insqr*(ndelt-sin)],
                            [-2*insqr*(ndelt-sin),  4*insqr*(1-cos)-3/2*deltaT**2],
                            [insqr*sin,             2*insqr*(1-cos)],
                            [-2*insqr*(1-cos),       4*insqr*sin - 3 * deltaT]])

        print(self.Ad)
        print(self.Bd)



class Chaser(Base):
    def __init__(self, r0=300000, deltaT=1):
        Base.__init__(self, r0,deltaT)

In [21]:
class Controller():
    def __init__(self, A, B, Q, R, Qf, umin, umax, dt):
        ##### INPUTS: 
        # A: a numpy array of shape (m, m) representing the A matrix in discrete LTI state space form
        # B: a numpy array of shape (m, n) representing the B matrix in discrete LTI state space form
        # Q: 
        # R: 
        # Qf: 
        # umin: a numpy array of shape (m, 1) describing the min thrust the chaser is capable of (described in the frame of the chaser)
        # umax: a numpy array of shape (m, 1) describing the max thrust the chaser is capable of (described in the frame of the chaser)
        # dt: 
        self.A = A
        self.B = B
        self.m = A.shape[0]
        self.n = B.shape[1]
        self.Q = Q
        self.R = R
        self.Qf = Qf
        self.umin = umin
        self.umax = umax
        self.dt = dt
        
        ##### Run LQR to find P and K
        P_now = Qf
        K_now = np.zeros((self.n,1))
        P_prev = np.zeros((self.m,self.m))
        K_prev = np.zeros((self.n,1))

        while not np.allclose(P_now,P_prev):
            P_now = np.copy(P_prev)
            K_now = np.copy(K_prev)
            P_prev = self.Q + np.dot(np.transpose(K_now),np.dot(self.R,K_now)) + np.transpose(self.A - np.dot(self.B, K_now))*P_now*(self.A - np.dot(self.B, K_now))
            K_prev = np.dot(np.linalg.inv(self.R + np.dot(np.transpose(self.B), np.dot(P_now, self.B))), np.dot(np.transpose(self.B), np.dot(P_now, self.A)))
        self.K = K_prev
        self.P = P_prev
        
        
    def lqr(xhat, P, Np, Nc, rport, omega):
        ##### INPUTS: 
        # xhat: a numpy array of shape (n, 1) representing the current state estimate (mean)
        # P: a numpy array of shape (n, n) representing the covariance uncertainty on the state estimate
        # Np: an int representing the prediction horizon (steps)
        # Nc: an int representing the control horizon (steps)
        # rport: a numpy array of shape (n/2,1) describing the x,y 
        
        ##### OUTPUTS: 
        # xpredictions: a numpy array of shape (Np + 1, m)
        # upredictions: a numpy array of shape (Np, n) representing the sequence of control inputs that is optimal over the prediction horizon, Np
        # u: a numpy array of shape (Nc, 2) representing the sequence of control inputs that is optimal over the control horizon, Nc. This is used to determine the next Nc control inputs

        # initialize the xpredictions matrix
        xpredictions = np.zeros((Np,self.m))
        # initalize upredictions
        upredictions = np.zeros((Np,self.n))
        # initialize u
        u = np.zeros((Nc,self.n))
        # initialize the array of states of the docking port for tracking
        xport = np.zeros((Np, m))
        # populate xport with the evolution of the docking port state over time including in [x,y,xdot,ydot]
        
                            
        # calculate the optimal u over Np by u = K_t x_t

        # populate the optimal u over Nc

        # use the optimal outputs over Np to populate xprediction


        return xprediction, upredictions, u




In [ ]:
        ##### Run LQR to find P and K
        # for a description of the algorithm, see pg 23 of https://stanford.edu/class/ee363/lectures/dlqr.pdf
        
        # initialize array of Pn for the LQR
#         Pn = np.zeros(Np, m, m)
#         Pn[Np-1,:,:] = Qf
        # iterate backwards from N,...,1 and populate P by the equation for P_{t-1}

        # iterate forwards from 0,...,N-1 and define K_t
#         self.Ks = np.zeros(Np, m, m)

In [22]:
base = Base(deltaT = 0.01)

0.12154255465717591 398860000000000.0 300000
[[ 1.00000222e+00  0.00000000e+00  8.22756494e+00  1.21542540e-05]
 [-1.79549851e-09  1.00000000e+00 -1.21542540e-05  9.99999015e-03]
 [ 4.43177669e-04  0.00000000e+00  9.99999261e-01  2.43085049e-03]
 [ 4.43177723e-06  0.00000000e+00 -2.43085049e-03  9.99997045e-01]]
[[ 4.99999938e-05  4.05141819e-08]
 [-4.05141819e-08  4.99999754e-05]
 [ 8.22756899e-02  9.99999877e-05]
 [-9.99999877e-05  2.99102760e-01]]


In [23]:
Q = np.eye(base.Ad.shape[0])
R = 10 * np.eye(base.Bd.shape[1])
Qf = 3 * np.eye(base.Ad.shape[0])
umin = -1.0 * np.ones((2,1))
umax = np.ones((2,1))
ctrl = Controller(base.Ad, base.Bd, Q, R, Qf, umin, umax)

In [ ]:
def fitSpiral(xpred, omega, r0):
    ##### INPUTS:
    # xpred: sequence of predictions to fit the Archimedes Spiral to
    # omega: 
    # r0: 
    ##### OUTPUTS:
    # b
    # c

In [ ]:
def mapFeasibility():
    pass